In [14]:
import pandas as pd
import numpy as np

In [15]:
data = pd.read_excel("tumor-data-without-healthy.xlsx")

In [16]:
labels = [value for column, value in data.iloc[0].items()]
data = data.set_axis(labels, axis=1)
data = data[data.columns[1:]].apply(
    lambda x: pd.to_numeric(x.replace('<', '').replace(',', '.'), errors='coerce')
)
data.drop(index=0, inplace=True)

In [17]:
data["Гормональная активность 0-нет               1-да"] = data[
    "Гормональная активность 0-нет               1-да"].astype(str)
data["Гормональная активность 0-нет               1-да"] = data[
"Гормональная активность 0-нет               1-да"].fillna("missing")

In [18]:
""" dropping some columns """

data.dropna(axis=1, how="all", inplace=True)
data.dropna(axis=0, thresh=int(data.shape[1] * 0.6), inplace=True)
data.dropna(axis=1, thresh=int(data.shape[0] * 0.6), inplace=True)
data.drop(["Пол:    0-жен, 1-муж"], inplace=True, axis=1)

In [19]:
X = data.drop(columns=["КАН", "АКР"])
y = np.argmax(data[["КАН", "АКР"]], axis=1)

In [20]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, random_state=42)

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer

pipeline = make_pipeline(
    SimpleImputer(strategy="median"),
    LogisticRegression(max_iter=1000, random_state=42)
)

In [22]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for i, (train_index, test_index) in enumerate(skf.split(X_train, y_train)):
    print(f"Fold {i}:")
    xtrain, xtest = X_train.iloc[train_index], X_train.iloc[test_index]
    ytrain, ytest = y_train[train_index], y_train[test_index]
    pipeline.fit(xtrain, ytrain)

    y_pred = pipeline.predict(xtest)

    print("Accuracy:", accuracy_score(ytest, y_pred))
    print("F1:", f1_score(ytest, y_pred))
    print("Precision:", precision_score(ytest, y_pred))
    print("Recall:", recall_score(ytest, y_pred))
    print()

Fold 0:
Accuracy: 0.7368421052631579
F1: 0.7058823529411765
Precision: 0.6666666666666666
Recall: 0.75

Fold 1:
Accuracy: 0.7777777777777778
F1: 0.7777777777777778
Precision: 0.7
Recall: 0.875

Fold 2:
Accuracy: 0.7777777777777778
F1: 0.75
Precision: 0.75
Recall: 0.75

Fold 3:
Accuracy: 0.7777777777777778
F1: 0.7142857142857143
Precision: 0.8333333333333334
Recall: 0.625

Fold 4:
Accuracy: 0.7777777777777778
F1: 0.7777777777777778
Precision: 0.7
Recall: 0.875



In [23]:
from sklearn.model_selection import cross_validate

cv = cross_validate(pipeline, X, y, n_jobs=-1,
                    scoring=["accuracy",
                             "f1",
                             "precision",
                             "recall"])

In [24]:
for i in range(len(cv)-1):
    print(
        " accuracy", cv["test_accuracy"][i], "\n",
        "f1", cv["test_f1"][i], "\n",
        "recall", cv["test_recall"][i], "\n",
        "precision", cv["test_precision"][i], "\n"
    )
    print("---------------------------------------------")

 accuracy 0.9523809523809523 
 f1 0.9523809523809523 
 recall 1.0 
 precision 0.9090909090909091 

---------------------------------------------
 accuracy 0.8571428571428571 
 f1 0.8235294117647058 
 recall 0.7 
 precision 1.0 

---------------------------------------------
 accuracy 0.75 
 f1 0.7368421052631579 
 recall 0.7777777777777778 
 precision 0.7 

---------------------------------------------
 accuracy 0.75 
 f1 0.7368421052631579 
 recall 0.7777777777777778 
 precision 0.7 

---------------------------------------------
 accuracy 0.6 
 f1 0.6363636363636364 
 recall 0.7777777777777778 
 precision 0.5384615384615384 

---------------------------------------------


In [25]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = pipeline.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))

print(classification_report(y_test, y_pred))

Accuracy: 0.9090909090909091
              precision    recall  f1-score   support

           0       0.80      1.00      0.89         4
           1       1.00      0.86      0.92         7

    accuracy                           0.91        11
   macro avg       0.90      0.93      0.91        11
weighted avg       0.93      0.91      0.91        11



In [26]:
model = pipeline["logisticregression"]

feature_importances = model.coef_.T.reshape(55,)
feature_names = X.columns

importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

importance_df

,Feature,Importance
15,dA2_17B,0.040020
10,свободный кортизон мочи (ВЭЖХ),0.038283
27,dP3_3А,0.025637
32,THS,0.022147
44,HHB,0.020650
5,Кортизол крови вечер,0.018510
17,16DHEA-3a,0.017912
2,"Нативная плотность, НU",0.017459
35,alloTHB,0.016631
14,Et,0.016085
